# Compatibilizador de malhas censitárias
---

### Importações

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

import os
import geopandas as gpd
import pandas as pd
import networkx as nx
from shapely import LineString, Polygon, MultiPolygon, distance, intersects, minimum_bounding_radius as min_radius
from shapely.geometry import box
from shapely.wkt import loads, dumps

In [ ]:
### Célula para conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

if not os.getcwd().endswith('Censo IBGE 2022/Compatibilização'):
    os.chdir('/content/drive/Shareddrives/SIG LabCidade/projetos/Censo IBGE 2022/Compatibilização')

Mounted at /content/drive


In [ ]:
nome_compat = '2000-2010-2022'
nome_A = '2000'
nome_B = '2010'
nome_C = '2022'

In [ ]:
if not os.path.isdir(nome_compat):
    os.mkdir(nome_compat)

### Funções gerais

In [ ]:
UTMCODES = {
    '17S':"EPSG:31977",
    '18S':"EPSG:31978",
    '19S':"EPSG:31979",
    '20S':"EPSG:31980",
    '21S':"EPSG:31981",
    '22S':"EPSG:31982",
    '23S':"EPSG:31983",
    '24S':"EPSG:31984",
    '25S':"EPSG:31985",
    '17N':"EPSG:31971",
    '18N':"EPSG:31972",
    '19N':"EPSG:31973",
    '20N':"EPSG:31974",
    '21N':"EPSG:31975",
    '22N':"EPSG:31976",
    '23N':"EPSG:6210",
    '24N':"EPSG:6211"
    }

# Função para identificar projeção local UTM
def find_utm_proj(X, Y):
    # Hemisfério
    h = 'N' if Y > 0 else 'S'

    # Fuso
    if  -84 <= X < -78:
        f = '17'
    elif  -78 <= X < -72:
        f = '18'
    elif  -72 <= X < -66:
        f = '19'
    elif  -66 <= X < -60:
        f = '20'
    elif  -60 <= X < -54:
        f = '21'
    elif  -54 <= X < -48:
        f = '22'
    elif  -48 <= X < -42:
        f = '23'
    elif  -42 <= X < -36:
        f = '24'
    elif  -36 <= X < -30:
        f = '25'
    else:
        f=''

    fuse = f'{f}{h}'
    return UTMCODES[fuse]

In [ ]:
# Função/regra para avaliar se geometria deve ser considerada expúria
def geomNotEspuria(geom):
    ap_ratio = geom.area/geom.length
    return (ap_ratio > 0.5 or geom.area > 500)

### Seleção de malhas

In [ ]:
# Seleção para cruzamento
muns = ['3503901', '3505708','3506359','3506607','3508405','3509007','3509205','3509601','3510609','3513009','3513504','3513801','3515004','3515103','3515707','3516309','3516408','3518305','3518701','3518800','3522109','3522208','3522505','3523107','3524006','3525003','3525201','3525904','3526209','3527306','3528502','3529401','3530607','3531100','3534401','3537602','3539103','3539806','3541000','3543303','3544103','3545001','3546801','3547304','3547809','3548500','3548708','3548807','3549953','3550308','3551009','3552502','3552809','3556453','3556503']
#muns = ['3550308']

In [ ]:
# Leitura das malhas
matrizes = {}
matrizes['2000-PC1'] = pd.read_csv('2000-2010-RMSP-RMBS-AUJ/matriz_compat_2000.csv', sep='\t', dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})
matrizes['2010-PC1'] = pd.read_csv('2000-2010-RMSP-RMBS-AUJ/matriz_compat_2010.csv', sep='\t', dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})
matrizes['2010-PC2'] = pd.read_csv('2010-2022-RMSP-RMBS-AUJ/matriz_compat_2010.csv', sep='\t', dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})
matrizes['2022-PC2'] = pd.read_csv('2010-2022-RMSP-RMBS-AUJ/matriz_compat_2022.csv', sep='\t', dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})

In [ ]:
for nome, df in matrizes.items():
    df = df.dropna()
    df['mun'] = df['CD_GEOCODI'].apply(lambda x: x[:7])
    matrizes[nome] = df.query('mun in @muns')
    matrizes[nome] = df.query('mun in @muns')

### Arquivo GPKG malha C

In [ ]:
gdf = gpd.read_file('Setores IBGE.gpkg', layer='SP_2022P')
gdf = gdf.rename(columns={'CD_SETOR':'CD_GEOCODI'})
gdf = gdf[gdf['CD_GEOCODI'].apply(lambda x: str(x)[:7] in muns)]

# Remoção do sufixo P
gdf['CD_GEOCODI'] = gdf['CD_GEOCODI'].apply(lambda x: x.replace('P',''))

### Campo GROUP representa municípios
for gdf in [gdf]:
    gdf['geometry'] = gdf['geometry'].make_valid()
    gdf['GROUP'] = gdf['CD_GEOCODI'].apply(lambda x: str(x)[:11])

In [ ]:
Y = (gdf.total_bounds[1] + gdf.total_bounds[3])/2
X = (gdf.total_bounds[0] + gdf.total_bounds[2])/2
UTMCRS = find_utm_proj(X, Y)
gdf = gdf.to_crs(UTMCRS)

## Construção de grafo

In [ ]:
G_compat = nx.Graph()

# Adicionar todos os nós
for nome, df in matrizes.items():
    X, Y = nome.split('-')
    for i, row in df.iterrows():
        # Adicionar nós
        G_compat.add_node(f"{X}.{row['CD_GEOCODI']}",
                        malha = X,
                        group = row['CD_GEOCODI'][:11],
                        nome=row['CD_GEOCODI'])
        G_compat.add_node(f"{Y}.{row['CD_PERIMETRO']}",
                        malha = Y,
                        group = row['CD_GEOCODI'][:11],
                        nome=row['CD_PERIMETRO'])
        # Adicionar arestas
        G_compat.add_edge(f"{X}.{row['CD_GEOCODI']}",
                      f"{Y}.{row['CD_PERIMETRO']}",
                      metodo=nome)


In [ ]:
# Limpar arestas entre grupos
to_remove = []
for u, v, d in G_compat.edges(data=True):
    group_A = G_compat.nodes[u]['group']
    group_B = G_compat.nodes[v]['group']
    if group_A != group_B:
        to_remove.append([u, v])

for u, v in to_remove:
    # Remove as arestas entre grupos apenas de nós que tem vínculos válidos
    if not all([e in to_remove for e in G_compat.edges(u)]) and not all([e in to_remove for e in G_compat.edges(v)]):
        G_compat.remove_edge(u,v)

In [ ]:
try:
    assert not list(nx.isolates(G_compat))
except AssertionError:
    print(len(list(nx.isolates(G_compat))), 'nós não conectados')

### Codificação das componentes do grafo de compatibilização

In [ ]:
componentes = [{'group':G_compat.nodes[list(G)[0]]['group'], 'nodes':list(G)}\
               for G in nx.connected_components(G_compat)]
increment_id = {k['group']:0 for k in componentes}

matriz_A = []
matriz_B = []
matriz_C = []
for c in componentes:
    increment_id[c['group']] += 1
    cod_c = f"{c['group']}{increment_id[c['group']]:05d}"

    c_A = [G_compat.nodes[i]['nome'] for i in c['nodes'] if G_compat.nodes[i]['malha']=='2000']
    matriz_A.append({'CD_PERIMETRO':cod_c, 'CD_GEOCODI':c_A})
    c_B = [G_compat.nodes[i]['nome'] for i in c['nodes'] if G_compat.nodes[i]['malha']=='2010']
    matriz_B.append({'CD_PERIMETRO':cod_c, 'CD_GEOCODI':c_B})
    c_C = [G_compat.nodes[i]['nome'] for i in c['nodes'] if G_compat.nodes[i]['malha']=='2022']
    matriz_C.append({'CD_PERIMETRO':cod_c, 'CD_GEOCODI':c_C})

# Criação dos DataFrames finais
df_matriz_A = pd.DataFrame(matriz_A)
df_matriz_A = df_matriz_A.explode('CD_GEOCODI')
df_matriz_B = pd.DataFrame(matriz_B)
df_matriz_B = df_matriz_B.explode('CD_GEOCODI')
df_matriz_C = pd.DataFrame(matriz_C)
df_matriz_C = df_matriz_C.explode('CD_GEOCODI')

### Exportação de matrizes de compatibilidade

In [ ]:
df_matriz_A[['CD_GEOCODI', 'CD_PERIMETRO']].to_csv(f'{nome_compat}/matriz_compat_{nome_A}.csv', sep='\t', index=False)
df_matriz_B[['CD_GEOCODI', 'CD_PERIMETRO']].to_csv(f'{nome_compat}/matriz_compat_{nome_B}.csv', sep='\t', index=False)
df_matriz_C[['CD_GEOCODI', 'CD_PERIMETRO']].to_csv(f'{nome_compat}/matriz_compat_{nome_C}.csv', sep='\t', index=False)

In [ ]:
# Contagem de membros A dos perímetros
data_matriz_A = df_matriz_A.pivot_table(index='CD_PERIMETRO',
                                        values='CD_GEOCODI',
                                        aggfunc='count').reset_index()
data_matriz_A = data_matriz_A.rename(columns={'CD_GEOCODI':'membros_A'})
# Contagem de membros B dos perímetros
data_matriz_B = df_matriz_B.pivot_table(index='CD_PERIMETRO',
                                        values='CD_GEOCODI',
                                        aggfunc='count').reset_index()
data_matriz_B = data_matriz_B.rename(columns={'CD_GEOCODI':'membros_B'})
# Contagem de membros C dos perímetros
data_matriz_C = df_matriz_C.pivot_table(index='CD_PERIMETRO',
                                        values='CD_GEOCODI',
                                        aggfunc='count').reset_index()
data_matriz_C = data_matriz_C.rename(columns={'CD_GEOCODI':'membros_C'})
# Agregação dos dados
data_matrizes = data_matriz_A.merge(data_matriz_B, on='CD_PERIMETRO')
data_matrizes = data_matrizes.merge(data_matriz_C, on='CD_PERIMETRO')
data_matrizes['membros'] = data_matrizes['membros_A'] + data_matrizes['membros_B'] + data_matrizes['membros_C']

# Validação
data_matrizes['valido'] = data_matrizes.apply(lambda x: all([x[i]>0 for i in ['membros_A', 'membros_B', 'membros_C']]), axis=1)

### Geopackage de perímetros compatíveis

In [38]:
def removeHoles(geom, area_min=1):
    if isinstance(geom, Polygon):
        geom = MultiPolygon([geom])
    out_polys = []
    for part in geom.geoms:
        interiors = []
        for i in part.interiors:
            p = Polygon(i)
            if p.area > area_min:
                interiors.append(i)
        out_polys.append(Polygon(part.exterior.coords, holes=interiors))
    return MultiPolygon(out_polys) if len(out_polys)>1 else out_polys[0]

gdf_perim_compat = df_matriz_C.merge(gdf, on='CD_GEOCODI')
gdf_perim_compat = gdf_perim_compat.merge(data_matrizes, on='CD_PERIMETRO')
gdf_perim_compat = gpd.GeoDataFrame(gdf_perim_compat, geometry='geometry', crs=UTMCRS)
gdf_perim_compat = gdf_perim_compat[['CD_PERIMETRO', 'GROUP', 'membros', 'membros_A', 'membros_B', 'membros_C', 'valido', 'geometry']].dissolve(by='CD_PERIMETRO')
gdf_perim_compat = gdf_perim_compat.rename(columns={'GROUP':'CD_DIST'})
gdf_perim_compat['CD_MUN'] = gdf_perim_compat['CD_DIST'].apply(lambda x: x[:7])
gdf_perim_compat['geometry'] = gdf_perim_compat['geometry'].apply(removeHoles)
gdf_perim_compat.to_file(f'{nome_compat}/perimetros_compativeis.gpkg',
                         layer=f'{nome_A}-{nome_B}-{nome_C}',
                         driver='GPKG')